# 1. Loading the Required Libraries

In [6]:
import numpy as np
import pandas as pd
import string
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings # supress warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)
import nltk
from nltk.corpus import stopwords

# 2. Loading the data

In [2]:
df1 =pd.read_json('dev.json')
df2 =pd.read_json('train_spider.json')
df3 =pd.read_json('train_others.json')

# 3. Summary of the Three dataset

In [ ]:
df1.info() # contains 1034 entries of 20 databases

In [ ]:
df2.info() # contains 7000 entries of 140 databases

In [ ]:
df3.info() # contains 1659 entries of 6 databases

In [ ]:
df1.head()

In [ ]:
df1.db_id.value_counts()

In [ ]:
df2.db_id.value_counts()

In [ ]:
df3.db_id.value_counts()

In [ ]:
# We will consider dev.json data for training purpose.

# 4. Data Pre-processing

Following are the major pre-processing steps used in NLP

•	Removing punctuations e.g ( ) * % @ ! $ .  ,
•	Removing Whitespaces
•	Lower casing
•	Removing URLs
•	Removing Stop words
•	Tokenization
•	Stemming
•	Lemmatization

We need to perform the above mentioned pre-processing steps based on the dataset.

In [3]:
# 4.1 Punctuations Removal
import string

def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text


#df1["clean_query"] = df1['query'].apply(remove_punctuations)
df1["clean_question"] = df1['question'].apply(remove_punctuations)
#df1['clean_question'] = df1.question.str.strip(string.punctuation)

In [4]:
df1.head()

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql,clean_question
0,concert_singer,SELECT count(*) FROM singer,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]",How many singers do we have?,"[How, many, singers, do, we, have, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[3, [0, [0, 0, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",How many singers do we have
1,concert_singer,SELECT count(*) FROM singer,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]",What is the total number of singers?,"[What, is, the, total, number, of, singers, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[3, [0, [0, 0, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",What is the total number of singers
2,concert_singer,"SELECT name , country , age FROM singer ORDER BY age DESC","[SELECT, name, ,, country, ,, age, FROM, singer, ORDER, BY, age, DESC]","[select, name, ,, country, ,, age, from, singer, order, by, age, desc]","Show name, country, age for all singers ordered by age from the oldest to the youngest.","[Show, name, ,, country, ,, age, for, all, singers, ordered, by, age, from, the, oldest, to, the, youngest, .]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 9, False], None]], [0, [0, [0, 10, False], None]], [0, [0, [0, 13, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': ['desc', [[0, [0, 13, False], None]]], 'limit': None, 'intersect': None, 'union': None, 'except': None}",Show name country age for all singers ordered by age from the oldest to the youngest
3,concert_singer,"SELECT name , country , age FROM singer ORDER BY age DESC","[SELECT, name, ,, country, ,, age, FROM, singer, ORDER, BY, age, DESC]","[select, name, ,, country, ,, age, from, singer, order, by, age, desc]","What are the names, countries, and ages for every singer in descending order of age?","[What, are, the, names, ,, countries, ,, and, ages, for, every, singer, in, descending, order, of, age, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 9, False], None]], [0, [0, [0, 10, False], None]], [0, [0, [0, 13, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': ['desc', [[0, [0, 13, False], None]]], 'limit': None, 'intersect': None, 'union': None, 'except': None}",What are the names countries and ages for every singer in descending order of age
4,concert_singer,"SELECT avg(age) , min(age) , max(age) FROM singer WHERE country = 'France'","[SELECT, avg, (, age, ), ,, min, (, age, ), ,, max, (, age, ), FROM, singer, WHERE, country, =, 'France, ']","[select, avg, (, age, ), ,, min, (, age, ), ,, max, (, age, ), from, singer, where, country, =, value]","What is the average, minimum, and maximum age of all singers from France?","[What, is, the, average, ,, minimum, ,, and, maximum, age, of, all, singers, from, France, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[5, [0, [0, 13, False], None]], [2, [0, [0, 13, False], None]], [1, [0, [0, 13, False], None]]]], 'where': [[False, 2, [0, [0, 10, False], None], '""France""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",What is the average minimum and maximum age of all singers from France


In [8]:
# 4.2 Whitespace Removal
df1['clean_query'] = df1['query'].str.strip(string.whitespace)
df1['clean_question'] = df1.clean_question.str.strip(string.whitespace)

In [11]:
df1.head()

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql,clean_question,clean_query
0,concert_singer,SELECT count(*) FROM singer,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]",How many singers do we have?,"[How, many, singers, do, we, have, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[3, [0, [0, 0, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",How many singers do we have,SELECT count(*) FROM singer
1,concert_singer,SELECT count(*) FROM singer,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]",What is the total number of singers?,"[What, is, the, total, number, of, singers, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[3, [0, [0, 0, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",What is the total number of singers,SELECT count(*) FROM singer
2,concert_singer,"SELECT name , country , age FROM singer ORDER BY age DESC","[SELECT, name, ,, country, ,, age, FROM, singer, ORDER, BY, age, DESC]","[select, name, ,, country, ,, age, from, singer, order, by, age, desc]","Show name, country, age for all singers ordered by age from the oldest to the youngest.","[Show, name, ,, country, ,, age, for, all, singers, ordered, by, age, from, the, oldest, to, the, youngest, .]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 9, False], None]], [0, [0, [0, 10, False], None]], [0, [0, [0, 13, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': ['desc', [[0, [0, 13, False], None]]], 'limit': None, 'intersect': None, 'union': None, 'except': None}",Show name country age for all singers ordered by age from the oldest to the youngest,"SELECT name , country , age FROM singer ORDER BY age DESC"
3,concert_singer,"SELECT name , country , age FROM singer ORDER BY age DESC","[SELECT, name, ,, country, ,, age, FROM, singer, ORDER, BY, age, DESC]","[select, name, ,, country, ,, age, from, singer, order, by, age, desc]","What are the names, countries, and ages for every singer in descending order of age?","[What, are, the, names, ,, countries, ,, and, ages, for, every, singer, in, descending, order, of, age, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 9, False], None]], [0, [0, [0, 10, False], None]], [0, [0, [0, 13, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': ['desc', [[0, [0, 13, False], None]]], 'limit': None, 'intersect': None, 'union': None, 'except': None}",What are the names countries and ages for every singer in descending order of age,"SELECT name , country , age FROM singer ORDER BY age DESC"
4,concert_singer,"SELECT avg(age) , min(age) , max(age) FROM singer WHERE country = 'France'","[SELECT, avg, (, age, ), ,, min, (, age, ), ,, max, (, age, ), FROM, singer, WHERE, country, =, 'France, ']","[select, avg, (, age, ), ,, min, (, age, ), ,, max, (, age, ), from, singer, where, country, =, value]","What is the average, minimum, and maximum age of all singers from France?","[What, is, the, average, ,, minimum, ,, and, maximum, age, of, all, singers, from, France, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[5, [0, [0, 13, False], None]], [2, [0, [0, 13, False], None]], [1, [0, [0, 13, False], None]]]], 'where': [[False, 2, [0, [0, 10, False], None], '""France""', None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",What is the average minimum and maximum age of all singers from France,"SELECT avg(age) , min(age) , max(age) FROM singer WHERE country = 'France'"


In [12]:
# 4.3 Lowering the text
df1['query_lower']= df1['clean_query'].apply(lambda x: x.lower())
df1['quest_lower']= df1['clean_question'].apply(lambda x: x.lower())
df1.head()

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql,clean_question,clean_query,query_lower,quest_lower
0,concert_singer,SELECT count(*) FROM singer,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]",How many singers do we have?,"[How, many, singers, do, we, have, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[3, [0, [0, 0, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",How many singers do we have,SELECT count(*) FROM singer,select count(*) from singer,how many singers do we have
1,concert_singer,SELECT count(*) FROM singer,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]",What is the total number of singers?,"[What, is, the, total, number, of, singers, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[3, [0, [0, 0, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",What is the total number of singers,SELECT count(*) FROM singer,select count(*) from singer,what is the total number of singers
2,concert_singer,"SELECT name , country , age FROM singer ORDER BY age DESC","[SELECT, name, ,, country, ,, age, FROM, singer, ORDER, BY, age, DESC]","[select, name, ,, country, ,, age, from, singer, order, by, age, desc]","Show name, country, age for all singers ordered by age from the oldest to the youngest.","[Show, name, ,, country, ,, age, for, all, singers, ordered, by, age, from, the, oldest, to, the, youngest, .]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 9, False], None]], [0, [0, [0, 10, False], None]], [0, [0, [0, 13, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': ['desc', [[0, [0, 13, False], None]]], 'limit': None, 'intersect': None, 'union': None, 'except': None}",Show name country age for all singers ordered by age from the oldest to the youngest,"SELECT name , country , age FROM singer ORDER BY age DESC","select name , country , age from singer order by age desc",show name country age for all singers ordered by age from the oldest to the youngest
3,concert_singer,"SELECT name , country , age FROM singer ORDER BY age DESC","[SELECT, name, ,, country, ,, age, FROM, singer, ORDER, BY, age, DESC]","[select, name, ,, country, ,, age, from, singer, order, by, age, desc]","What are the names, countries, and ages for every singer in descending order of age?","[What, are, the, names, ,, countries, ,, and, ages, for, every, singer, in, descending, order, of, age, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 9, False], None]], [0, [0, [0, 10, False], None]], [0, [0, [0, 13, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': ['desc', [[0, [0, 13, False], None]]], 'limit': None, 'intersect': None, 'union': None, 'except': None}",What are the names countries and ages for every singer in descending order of age,"SELECT name , country , age FROM singer ORDER BY age DESC","select name , country , age from singer order by age desc",what are the names countries and ages for every singer in descending order of age
4,concert_singer,"SELECT avg(age) , min(age) , max(age) FROM singer WHERE country = 'France'","[SELECT, avg, (, age, ), ,, min, (, age, ), ,, max, (, age, ), FROM, singer, WHERE, country, =, 'France, ']","[select, avg, (, age, ), ,, min, (, age, ), ,, max, (, age, ), from, singer, where, country, =, value]","What is the average, minimum, and maximum age of all singers from France?","[What, is, the, average, ,, minimum, ,, and, maximum, age, of, all, singers, from, France, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[5, [0, [0, 13, False], None]], [2, [0, [0, 13, False], None]], [1, [0, [0, 13, False], None]]]], 'where': 

In [13]:
# 4.4 Tokenization
# This step splits the text into smaller units.
import re

def tokenization(text):
    tokens = re.split('\W+',text)
    return tokens

#applying function to the column
df1['query_tokenised']= df1['query_lower'].apply(lambda x: tokenization(x))
df1['quest_tokenised']= df1['quest_lower'].apply(lambda x: tokenization(x))


In [14]:
df1.head()

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql,clean_question,clean_query,query_lower,quest_lower,query_tokenised,quest_tokenised
0,concert_singer,SELECT count(*) FROM singer,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]",How many singers do we have?,"[How, many, singers, do, we, have, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[3, [0, [0, 0, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",How many singers do we have,SELECT count(*) FROM singer,select count(*) from singer,how many singers do we have,"[select, count, from, singer]","[how, many, singers, do, we, have]"
1,concert_singer,SELECT count(*) FROM singer,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]",What is the total number of singers?,"[What, is, the, total, number, of, singers, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[3, [0, [0, 0, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}",What is the total number of singers,SELECT count(*) FROM singer,select count(*) from singer,what is the total number of singers,"[select, count, from, singer]","[what, is, the, total, number, of, singers]"
2,concert_singer,"SELECT name , country , age FROM singer ORDER BY age DESC","[SELECT, name, ,, country, ,, age, FROM, singer, ORDER, BY, age, DESC]","[select, name, ,, country, ,, age, from, singer, order, by, age, desc]","Show name, country, age for all singers ordered by age from the oldest to the youngest.","[Show, name, ,, country, ,, age, for, all, singers, ordered, by, age, from, the, oldest, to, the, youngest, .]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 9, False], None]], [0, [0, [0, 10, False], None]], [0, [0, [0, 13, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': ['desc', [[0, [0, 13, False], None]]], 'limit': None, 'intersect': None, 'union': None, 'except': None}",Show name country age for all singers ordered by age from the oldest to the youngest,"SELECT name , country , age FROM singer ORDER BY age DESC","select name , country , age from singer order by age desc",show name country age for all singers ordered by age from the oldest to the youngest,"[select, name, country, age, from, singer, order, by, age, desc]","[show, name, country, age, for, all, singers, ordered, by, age, from, the, oldest, to, the, youngest]"
3,concert_singer,"SELECT name , country , age FROM singer ORDER BY age DESC","[SELECT, name, ,, country, ,, age, FROM, singer, ORDER, BY, age, DESC]","[select, name, ,, country, ,, age, from, singer, order, by, age, desc]","What are the names, countries, and ages for every singer in descending order of age?","[What, are, the, names, ,, countries, ,, and, ages, for, every, singer, in, descending, order, of, age, ?]","{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[0, [0, [0, 9, False], None]], [0, [0, [0, 10, False], None]], [0, [0, [0, 13, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': ['desc', [[0, [0, 13, False], None]]], 'limit': None, 'intersect': None, 'union': None, 'except': None}",What are the names countries and ages for every singer in descending order of age,"SELECT name , country , age FROM singer ORDER BY age DESC","select name , country , age from singer order by age desc",what are the names countries and ages for every singer in descending order of age,"[select, name, country, age, from, singer, order, by, age, desc]","[what, are, the, names, countries, and, ages, for, every, singer, in, descending, order, of, age]"
4,concert_singer,"SELECT avg(age) , min(age) , max(age) FROM singer WHERE country = 'France'","[SELECT, avg, (, age, ), ,, min, (, age, ), ,, max, (, a

In [ ]:
# 4.6 Lemmatization : We will use Lemmatization instead of stemming as stemming causes loss of word meaning .Lemmatization
# on the other hand don't lose the meaning of the words in the root form

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()

#defining the function for lemmatization
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text
df1['quest_lemmatized']=df1['quest_tokenised'].apply(lambda x:lemmatizer(x))

In [ ]:
df1.head()

In [ ]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

df.db_id.value_counts()

In [ ]:
# Whitespaces also could be considered in some cases.
import string

unwanted = string.punctuation + string.whitespace
print(unwanted)

df.question = df.question.str.strip(unwanted)

In [ ]:
df.question = df.question.str.lower()

In [ ]:
df.head()

In [ ]:
nltk.download('stopwords')
stopwords = stopwords.words('english')

In [ ]:
#df['db_id' == 'real_estate_properties'].head()

df[df['db_id'] == 'real_estate_properties']

In [ ]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]